# Fine-Tune a Text Classifier with Transformers
This notebook demonstrates how to fine-tune a pretrained DistilBERT model on the IMDb sentiment dataset.

## Install Required Libraries

In [ ]:
#!pip install transformers datasets --quiet

In [ ]:
#!pip install scikit-learn

In [ ]:
#!pip install --upgrade transformers

## Import Libraries

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
print('--------------------')
import transformers
print('--------------------')

## Load IMDb Dataset

In [ ]:
dataset = load_dataset('imdb')
train_data = dataset['train'].shuffle(seed=42).select(range(2000))  # Smaller subset for quick training
test_data = dataset['test'].shuffle(seed=42).select(range(500))

## Load Tokenizer and Tokenize the Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

train_enc = train_data.map(tokenize_function, batched=True)
test_enc = test_data.map(tokenize_function, batched=True)

## Load Pretrained Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

## Define Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds)
    }

## Set Training Arguments

In [ ]:
!python -c "import transformers; print(transformers.__version__)"

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch"
)


## Initialize Trainer and Train

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    compute_metrics=compute_metrics
)

trainer.train()

## Evaluate the Model

In [ ]:
trainer.evaluate()

## Save the Model

In [ ]:
model.save_pretrained("./sentiment-model")
tokenizer.save_pretrained("./sentiment-model")

In [ ]:
# from datasets import load_dataset
# import pandas as pd

# # Load IMDb dataset
# dataset = load_dataset('imdb')

# # Convert to DataFrames
# train_df = pd.DataFrame(dataset['train'])
# test_df = pd.DataFrame(dataset['test'])

# # Save using tab as separator
# train_df.to_csv('train.tsv', sep='\t', index=False)
# test_df.to_csv('test.tsv', sep='\t', index=False)

# print("Saved as 'train.tsv' and 'test.tsv' with tab separators.")


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the Arabic Sentiments dataset
dataset = load_dataset("ImranzamanML/Arabic-Sentiments")

# # Convert to pandas DataFrames
# train_df = pd.DataFrame(dataset["train"])
# test_df = pd.DataFrame(dataset["test"])

# # Optional: Save locally as TSV
# train_df.to_csv("arabic_train.tsv", sep="\t", index=False)
# test_df.to_csv("arabic_test.tsv", sep="\t", index=False)

# print("✅ Saved: arabic_train.tsv and arabic_test.tsv")


In [ ]:
train_df.shape

## Summary
- We fine-tuned a DistilBERT model on a subset of the IMDb dataset.
- We used Hugging Face `Trainer` for easy training and evaluation.
- The model is saved locally and ready for deployment or inference.